# Avaliação Estatística - Comparação dos 5 Braços Experimentais

Este notebook realiza análise estatística comparativa dos resultados dos 5 braços experimentais usando:

- **Teste de Friedman:** Para verificar se há diferenças significativas entre os braços
- **Pós-testes:** Nemenyi, Conover e Bonferroni para identificar quais braços diferem significativamente
- **Visualizações:** Rankings, heatmaps e diagramas de diferença crítica

## Braços Comparados

1. Baseline CNN
2. ViT Puro
3. ViT + Contrastive
4. ViT + MIM
5. ViT + Sparse


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.stats import friedmanchisquare, chi2, t, norm
from scikit_posthocs import posthoc_conover, posthoc_dunn, posthoc_nemenyi_friedman

# MLflow para rastreamento de experimentos
import mlflow
from mlflow import log_metric, log_param, log_artifacts

# ============================================
# DETECÇÃO DE AMBIENTE (COLAB OU LOCAL)
# ============================================
try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Colab detectado - Drive montado")
except ImportError:
    IN_COLAB = False
    print("✅ Ambiente local detectado")

# Configurar caminhos baseado no ambiente
if IN_COLAB:
    BASE_DIR = Path("/content/drive/MyDrive/Mestrado_TCC")
    RESULTS_DIR = BASE_DIR / "results" / "classifications"
    EVAL_DIR = BASE_DIR / "results" / "evaluations"
    MLRUNS_DIR = BASE_DIR / "mlruns"
    # Mudar para diretório do framework
    FRAMEWORK_DIR = BASE_DIR / "Framework"
    if FRAMEWORK_DIR.exists():
        os.chdir(FRAMEWORK_DIR)
else:
    BASE_DIR = Path("../")
    RESULTS_DIR = BASE_DIR / "results" / "classifications"
    EVAL_DIR = BASE_DIR / "results" / "evaluations"
    MLRUNS_DIR = BASE_DIR / "mlruns"

# Criar diretórios
EVAL_DIR.mkdir(parents=True, exist_ok=True)

# Configuração do MLflow
MLRUNS_DIR.mkdir(exist_ok=True)
mlflow.set_tracking_uri(str(MLRUNS_DIR.absolute()))

# Lista dos braços experimentais
EXPERIMENTAL_ARMS = [
    "baseline_cnn",
    "vit_pure",
    "vit_contrastive",
    "vit_mim",
    "vit_sparse"
]

print(f"\n📁 Diretórios configurados:")
print(f"   Resultados: {RESULTS_DIR}")
print(f"   Avaliações: {EVAL_DIR}")
print(f"   MLflow: {MLRUNS_DIR}")


## Classe StatisticalAnalysis

Classe adaptada do notebook original para análise estatística dos braços experimentais.


In [ ]:
class StatisticalAnalysis:
    def __init__(self, f1_scores_dict, alpha=0.05):
        """
        Classe para realizar Teste de Friedman e Pós-Testes de Diferença Crítica.
        
        Parâmetros:
        - f1_scores_dict: dicionário contendo os nomes dos braços e seus F1-scores por fold.
        - alpha: nível de significância estatística (padrão: 0.05).
        """
        self.f1_scores_dict = f1_scores_dict
        self.models = list(f1_scores_dict.keys())
        self.results_array = np.array(list(f1_scores_dict.values()))
        self.alpha = alpha
        self.N = len(next(iter(f1_scores_dict.values())))  # Número de folds
        self.k = len(self.models)  # Número de braços
        self.df_friedman = self.k - 1
        
        # Valores críticos
        self.valor_critico_friedman = chi2.ppf(1 - self.alpha, self.df_friedman)
        self.q_nemenyi = self.get_q_nemenyi(self.k)
        self.cd_nemenyi = self.q_nemenyi * np.sqrt(self.k * (self.k + 1) / (6 * self.N))
        
        print(f"Número de experimentos (N) = {self.N}")
        print(f"Número de braços (k) = {self.k}")
        print(f"Valor crítico de Friedman (α={self.alpha}) = {self.valor_critico_friedman:.3f}")
        print(f"Diferença Crítica (CD) para Nemenyi = {self.cd_nemenyi:.3f}")

    def get_q_nemenyi(self, k):
        """Obtém o valor crítico q_0.05 da tabela do pós-teste de Nemenyi."""
        nemenyi_table = {
            2: 1.960, 3: 2.343, 4: 2.569, 5: 2.728, 6: 2.850,
            7: 2.949, 8: 3.031, 9: 3.102, 10: 3.164
        }
        return nemenyi_table.get(k, 3.164)

    def friedman_test(self):
        """Executa o Teste de Friedman."""
        statistic, p_value = friedmanchisquare(*self.results_array)
        print(f"\nTeste de Friedman (F1-score):")
        print(f"  Estatística = {statistic:.4f}")
        print(f"  p-valor = {p_value:.4f}")
        
        if p_value < self.alpha:
            print(f"  **Rejeitamos H0** (p < {self.alpha}): Há diferença significativa entre os braços")
        else:
            print(f"  **Não rejeitamos H0** (p >= {self.alpha}): Não há diferença significativa")
        
        return p_value

    def nemenyi_test(self):
        """Executa o Pós-Teste de Nemenyi."""
        df_results = pd.DataFrame(self.results_array.T, columns=self.models)
        posthoc_res = posthoc_nemenyi_friedman(df_results)
        print("\nPós-Teste de Nemenyi:\n", posthoc_res)
        self.plot_cd_diagram(posthoc_res, "Nemenyi")
        return posthoc_res

    def plot_cd_diagram(self, posthoc_matrix, test_type="Nemenyi"):
        """Gera um Diagrama de Diferença Crítica (CD Diagram)."""
        f1_means = {model: np.mean(scores) for model, scores in self.f1_scores_dict.items()}
        rankings = sorted(f1_means.items(), key=lambda x: x[1], reverse=True)
        
        plt.figure(figsize=(10, 6))
        plt.title(f"Diagrama de Diferença Crítica ({test_type})", fontsize=14, fontweight='bold')
        
        y_pos = 1
        for i, (model, score) in enumerate(rankings):
            plt.scatter(score, y_pos, color="black", s=100, zorder=3)
            plt.text(score, y_pos + 0.15, model.replace('_', ' ').title(), 
                    ha='center', fontsize=10, fontweight='bold')
            y_pos += 1
        
        # Linha de Diferença Crítica
        if len(rankings) > 0:
            plt.plot([rankings[0][1] - self.cd_nemenyi, rankings[0][1] + self.cd_nemenyi], 
                    [y_pos - 1.5, y_pos - 1.5], color="red", linewidth=2, linestyle='--', zorder=2)
            plt.text(rankings[0][1], y_pos - 1.2, f"CD = {self.cd_nemenyi:.3f}", 
                    ha='center', fontsize=10, fontweight="bold", color='red')
        
        plt.xlabel("F1-Score Médio", fontsize=12)
        plt.ylabel("Ranking dos Braços", fontsize=12)
        plt.yticks([])
        plt.grid(axis='x', linestyle="--", alpha=0.5)
        plt.tight_layout()
        plt.savefig(EVAL_DIR / "cd_diagram.png", dpi=300, bbox_inches='tight')
        plt.show()

    def posthoc_tests(self, method="conover"):
        """Executa os pós-testes de Conover ou Bonferroni."""
        df_results = pd.DataFrame(self.f1_scores_dict)
        df_results = df_results.melt(var_name='group', value_name='value')
        
        if method == "conover":
            posthoc_res = posthoc_conover(df_results, val_col='value', group_col='group', p_adjust="holm")
            title = "Pós-Teste de Conover (Holm) - F1-score"
        elif method == "bonferroni":
            posthoc_res = posthoc_dunn(df_results, val_col='value', group_col='group', p_adjust="bonferroni")
            title = "Pós-Teste de Dunn (Bonferroni) - F1-score"
        else:
            raise ValueError("Método inválido. Escolha 'conover' ou 'bonferroni'.")
        
        print(f"\n{title}:\n", posthoc_res)
        self.plot_heatmap(posthoc_res, title)
        return posthoc_res

    def plot_heatmap(self, posthoc_matrix, title="Pós-Teste Estatístico"):
        """Gera um heatmap para visualizar os resultados do pós-teste."""
        plt.figure(figsize=(10, 8))
        sns.heatmap(posthoc_matrix, annot=True, cmap="RdYlGn_r", fmt=".4f", 
                   linewidths=0.5, center=0.05, vmin=0, vmax=1)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(EVAL_DIR / f"heatmap_{title.lower().replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
        plt.show()

    def plot_rankings(self):
        """Gera um gráfico de ranking dos braços baseado no F1-score médio."""
        f1_means = {model: np.mean(scores) for model, scores in self.f1_scores_dict.items()}
        sorted_f1 = sorted(f1_means.items(), key=lambda x: x[1], reverse=True)
        
        plt.figure(figsize=(10, 6))
        colors = sns.color_palette("viridis", len(sorted_f1))
        bars = plt.barh([x[0].replace('_', ' ').title() for x in sorted_f1], 
                        [x[1] for x in sorted_f1], color=colors)
        plt.xlabel("F1-Score Médio", fontsize=12)
        plt.ylabel("Braço Experimental", fontsize=12)
        plt.title("Ranking dos Braços Experimentais (F1-Score)", fontsize=14, fontweight='bold')
        
        # Adicionar valores nas barras
        for i, (bar, (model, score)) in enumerate(zip(bars, sorted_f1)):
            plt.text(score, i, f' {score:.4f}', va='center', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(EVAL_DIR / "rankings.png", dpi=300, bbox_inches='tight')
        plt.show()

print("Classe StatisticalAnalysis definida!")


## Carregar Resultados dos Classificadores

Carrega os resultados dos classificadores SVM e SRC para cada braço experimental.


In [ ]:
def load_classification_results(classifier_type="SVM"):
    """
    Carrega resultados de classificação para todos os braços.
    
    Args:
        classifier_type: "SVM" ou "SRC"
    
    Returns:
        dict: Dicionário com F1-scores por braço
    """
    f1_scores = {}
    
    for arm in EXPERIMENTAL_ARMS:
        result_path = RESULTS_DIR / f"{arm}_{classifier_type.lower()}_results.json"
        
        if not result_path.exists():
            print(f"⚠️  Resultados não encontrados para {arm} - {classifier_type}")
            continue
        
        with open(result_path, 'r') as f:
            results = json.load(f)
        
        # Para análise estatística, precisamos de múltiplos valores (folds)
        # Se tiver apenas um valor, vamos usar validação cruzada ou replicar
        # Por enquanto, vamos usar o valor de teste como exemplo
        # Em um cenário real, você teria múltiplos folds
        
        # Se os resultados tiverem múltiplos folds, use-os
        # Caso contrário, vamos simular múltiplos valores baseados no resultado único
        test_f1 = results.get('test_f1_macro', 0)
        
        # Para demonstração, vamos criar 5 valores similares (em produção, use folds reais)
        # Você pode modificar isso para usar validação cruzada real
        f1_scores[arm] = [test_f1] * 5  # Simulação - substitua por folds reais
        
        print(f"✅ {arm} - {classifier_type}: F1 = {test_f1:.4f}")
    
    return f1_scores

# Carregar resultados
print("Carregando resultados SVM...")
svm_f1_scores = load_classification_results("SVM")

print("\nCarregando resultados SRC...")
src_f1_scores = load_classification_results("SRC")

print(f"\n✅ {len(svm_f1_scores)} braços com resultados SVM")
print(f"✅ {len(src_f1_scores)} braços com resultados SRC")


## Análise Estatística - SVM

Executa análise estatística completa para os resultados do classificador SVM.


In [ ]:
if len(svm_f1_scores) >= 3:  # Mínimo de 3 braços para teste de Friedman
    print("="*60)
    print("ANÁLISE ESTATÍSTICA - CLASSIFICADOR SVM")
    print("="*60)
    
    analysis_svm = StatisticalAnalysis(svm_f1_scores, alpha=0.05)
    
    # Teste de Friedman
    p_value = analysis_svm.friedman_test()
    
    # Se houver diferença significativa, executar pós-testes
    if p_value < 0.05:
        print("\n" + "="*60)
        print("PÓS-TESTES (diferença significativa detectada)")
        print("="*60)
        
        # Teste de Nemenyi
        nemenyi_results = analysis_svm.nemenyi_test()
        nemenyi_results.to_csv(EVAL_DIR / "svm_nemenyi_results.csv")
        
        # Teste de Conover
        conover_results = analysis_svm.posthoc_tests("conover")
        conover_results.to_csv(EVAL_DIR / "svm_conover_results.csv")
        
        # Visualizações
        analysis_svm.plot_rankings()
    else:
        print("\n⚠️  Não há diferença significativa entre os braços (p >= 0.05)")
        print("   Pós-testes não são necessários.")
        analysis_svm.plot_rankings()
else:
    print("⚠️  Número insuficiente de braços para análise estatística (mínimo: 3)")


## Análise Estatística - SRC

Executa análise estatística completa para os resultados do classificador SRC.


In [ ]:
if len(src_f1_scores) >= 3:  # Mínimo de 3 braços para teste de Friedman
    print("="*60)
    print("ANÁLISE ESTATÍSTICA - CLASSIFICADOR SRC")
    print("="*60)
    
    analysis_src = StatisticalAnalysis(src_f1_scores, alpha=0.05)
    
    # Teste de Friedman
    p_value = analysis_src.friedman_test()
    
    # Se houver diferença significativa, executar pós-testes
    if p_value < 0.05:
        print("\n" + "="*60)
        print("PÓS-TESTES (diferença significativa detectada)")
        print("="*60)
        
        # Teste de Nemenyi
        nemenyi_results = analysis_src.nemenyi_test()
        nemenyi_results.to_csv(EVAL_DIR / "src_nemenyi_results.csv")
        
        # Teste de Conover
        conover_results = analysis_src.posthoc_tests("conover")
        conover_results.to_csv(EVAL_DIR / "src_conover_results.csv")
        
        # Visualizações
        analysis_src.plot_rankings()
    else:
        print("\n⚠️  Não há diferença significativa entre os braços (p >= 0.05)")
        print("   Pós-testes não são necessários.")
        analysis_src.plot_rankings()
else:
    print("⚠️  Número insuficiente de braços para análise estatística (mínimo: 3)")


## Comparação entre Classificadores

Compara o desempenho de SVM vs SRC para cada braço experimental.


In [ ]:
def compare_classifiers():
    """Compara SVM vs SRC para cada braço."""
    comparison_data = []
    
    for arm in EXPERIMENTAL_ARMS:
        svm_path = RESULTS_DIR / f"{arm}_svm_results.json"
        src_path = RESULTS_DIR / f"{arm}_src_results.json"
        
        svm_f1 = None
        src_f1 = None
        
        if svm_path.exists():
            with open(svm_path, 'r') as f:
                svm_results = json.load(f)
                svm_f1 = svm_results.get('test_f1_macro', None)
        
        if src_path.exists():
            with open(src_path, 'r') as f:
                src_results = json.load(f)
                src_f1 = src_results.get('test_f1_macro', None)
        
        comparison_data.append({
            'Braço': arm.replace('_', ' ').title(),
            'SVM F1': f"{svm_f1:.4f}" if svm_f1 else "N/A",
            'SRC F1': f"{src_f1:.4f}" if src_f1 else "N/A",
            'Melhor': 'SVM' if (svm_f1 and src_f1 and svm_f1 > src_f1) else ('SRC' if src_f1 else 'N/A')
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\nComparação SVM vs SRC:")
    print(comparison_df.to_string(index=False))
    
    # Salvar
    comparison_df.to_csv(EVAL_DIR / "classifier_comparison.csv", index=False)
    
    # Visualização
    if len(comparison_data) > 0:
        plt.figure(figsize=(12, 6))
        x = np.arange(len(comparison_data))
        width = 0.35
        
        svm_scores = [float(d['SVM F1']) if d['SVM F1'] != 'N/A' else 0 for d in comparison_data]
        src_scores = [float(d['SRC F1']) if d['SRC F1'] != 'N/A' else 0 for d in comparison_data]
        
        plt.bar(x - width/2, svm_scores, width, label='SVM', alpha=0.8)
        plt.bar(x + width/2, src_scores, width, label='SRC', alpha=0.8)
        
        plt.xlabel('Braço Experimental', fontsize=12)
        plt.ylabel('F1-Score', fontsize=12)
        plt.title('Comparação SVM vs SRC por Braço Experimental', fontsize=14, fontweight='bold')
        plt.xticks(x, [d['Braço'] for d in comparison_data], rotation=45, ha='right')
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(EVAL_DIR / "classifier_comparison.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return comparison_df

# Executar comparação
comparison_df = compare_classifiers()
